In [1]:
import os
import pandas as pd
from glob import glob
import json
import fnmatch
#import qgrid
import urllib.request, json
import datetime

## Check the ES-DOC errata pages for updates
- make 'csv/errata-files.csv' for using to make the updated catalog (MakeCloudCat.ipynb)
https://es-doc.github.io/esdoc-errata-client/installation.html
- many questions remain:
  - does status=resolved mean these files have been REPLACED? same version?
  - what do the various severity levels really mean? seems to be different for different models
  - could make a list of changed/new issues
  - could go through all 200+ issues and evaluate - feasible? make new boolean column 'include'
  - how to more efficiently deal with issues with >1,000 files, etc

# Sometimes the esgissue does NOT WORK, then will need to go back to old way:
- Use chrome to download 'Web Page single file' for each set of 100 issues into ES-DOC?.mht

/bin/grep "<tr id=" ES-DOC1.mht > issues.txt
    
/bin/grep "<tr id=" ES-DOC2.mht >> issues.txt

/bin/grep "<tr id=" ES-DOC3.mht >> issues.txt

In [25]:
# maybe we should delete contents of issues and dsets directories?
# os/system('rm -rf issues dsets')
#command = '/usr/local/python/anaconda3/envs/pangeo-Oct2019/bin/esgissue retrieve -i $PWD/issues -d $PWD/dsets'
command = 'esgissue retrieve -i $PWD/issues -d $PWD/dsets'
#os.system(command)

ifiles = glob('issues/issue*.json')
issues = [ifile.split('issue_')[1].split('.json')[0] for ifile in ifiles]
len(issues)

286

In [26]:
df = pd.DataFrame(columns=['uid', 'title', 'description', 'project', 'severity', 'status','urls'])
df_list = []
for item,issue in enumerate(issues):
    file_dsets = 'dsets/dset_'+issue+'.txt'
    if not os.path.isfile(file_dsets):
        file_dsets = 'dsets/dset_none.txt'

    file_issue = 'issues/issue_'+issue+'.json'
    
    with open(file_issue) as json_file:
        dict_issue = json.load(json_file)
   
    try:
        dlist = dict_issue['urls']
    except:
        dict_issue['urls'] = []
    
    df = df.append(dict_issue,ignore_index=True)
    print(item,file_dsets)
    
    df_dsets = pd.read_csv(file_dsets,delim_whitespace=True,header=None)
    df_dsets = df_dsets.rename(columns={0: "file_id"}).set_index([df_dsets.index])
    df_list += [list(df_dsets.file_id.values)]

df['file_ids'] = df_list
df = df.rename(columns={"uid": "issue_uid"})

0 dsets/dset_ee4e0a5c-d1e1-a67d-8640-722e761e7da8.txt
1 dsets/dset_99ffcdca-17dd-6843-9c42-511e28dce445.txt
2 dsets/dset_97c484f8-293f-95de-adb9-854b365c4951.txt
3 dsets/dset_5c00b84c-4622-689d-9a8d-e49d1a4acff6.txt
4 dsets/dset_780ffab8-ce57-e486-8994-32c5be093bff.txt
5 dsets/dset_4e6fbba4-6f27-9c2f-906e-ef699bb68dab.txt
6 dsets/dset_dca706d0-3a8a-a707-b350-a939122f9132.txt
7 dsets/dset_c628a0f9-0f74-942c-9583-dacbcd306469.txt
8 dsets/dset_f345199d-e417-98fe-a79d-e2f2fa8b0cd5.txt
9 dsets/dset_750a143a-5131-5631-a0ca-589273c817a1.txt
10 dsets/dset_b0e9b074-425e-0f90-9f98-bbf3be5913ea.txt
11 dsets/dset_d9ab69ba-2cfd-40a0-b3f2-0271f6cee3ff.txt
12 dsets/dset_172499d1-6553-8b9d-959d-879d33ff9cb6.txt
13 dsets/dset_8fbd8df5-c349-315b-9ec3-5a2f4ec4ec63.txt
14 dsets/dset_fedee017-40b6-fcb3-8fca-48f9730de0da.txt
15 dsets/dset_884a09c0-93ac-3da4-82ac-84c4025590c0.txt
16 dsets/dset_c36c71da-b998-8339-a107-6b8c1c5d5c9c.txt
17 dsets/dset_b57e7df0-bec0-941e-872a-dd4bb0d744a6.txt
18 dsets/dset_7c2396

In [27]:
df.to_csv('csv/ES-DOC_issues.csv', index=False)
df.issue_uid.nunique()

286

In [28]:
keywords = ['issue_uid','source_id', 'experiment_id', 'member_id', 'table_id', 'variable_id', 'grid_label', 'version', 'file_id','status','severity','issue_url']
df_all = []
for index, row in df.iterrows():
    if index > 143:
        continue
    file_id = row['file_ids']
    issue = row['issue_uid']
    status = row['status']
    severity = row['severity']
    issue_url = "https://errata.es-doc.org/static/view.html?uid="+issue
    dfs = pd.DataFrame(columns=keywords)
    print(index, ': issue uid:', issue, ', number of affected files:',len(file_id))
    for file in file_id:
        try:
            [fill,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_version] = file.split('.')
            [grid_label,version] = grid_version.split('#')
            klist = [issue,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,file,status,severity,issue_url]
            kdict = dict(zip(keywords, klist))
            dfs = dfs.append(kdict,ignore_index=True)
            df_all += [dfs]
        except:   # one issue has a list of datasets with two lines that need to be split apart
            #print('splitting lines for:',file_id)
            [s1,s2,s3]=file.split('CMIP6')
            files = ['CMIP6'+s2,'CMIP6'+s3]
            for f in files:
                [fill,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_version] = f.split('.')
                [grid_label,version] = grid_version.split('#')
                klist = [issue,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,f,status,severity,issue_url]
                kdict = dict(zip(keywords, klist))
                dfs = dfs.append(kdict,ignore_index=True)
                df_all += [dfs]
df_expand0 = pd.concat(df_all,sort=False).drop_duplicates(subset =["file_id"])

0 : issue uid: ee4e0a5c-d1e1-a67d-8640-722e761e7da8 , number of affected files: 158
1 : issue uid: 99ffcdca-17dd-6843-9c42-511e28dce445 , number of affected files: 50
2 : issue uid: 97c484f8-293f-95de-adb9-854b365c4951 , number of affected files: 112
3 : issue uid: 5c00b84c-4622-689d-9a8d-e49d1a4acff6 , number of affected files: 127
4 : issue uid: 780ffab8-ce57-e486-8994-32c5be093bff , number of affected files: 89
5 : issue uid: 4e6fbba4-6f27-9c2f-906e-ef699bb68dab , number of affected files: 10
6 : issue uid: dca706d0-3a8a-a707-b350-a939122f9132 , number of affected files: 4
7 : issue uid: c628a0f9-0f74-942c-9583-dacbcd306469 , number of affected files: 4
8 : issue uid: f345199d-e417-98fe-a79d-e2f2fa8b0cd5 , number of affected files: 1
9 : issue uid: 750a143a-5131-5631-a0ca-589273c817a1 , number of affected files: 6
10 : issue uid: b0e9b074-425e-0f90-9f98-bbf3be5913ea , number of affected files: 19
11 : issue uid: d9ab69ba-2cfd-40a0-b3f2-0271f6cee3ff , number of affected files: 14
12 

In [29]:
keywords = ['issue_uid','source_id', 'experiment_id', 'member_id', 'table_id', 'variable_id', 'grid_label', 'version', 'file_id','status','severity','issue_url']
df_all = []
for index, row in df.iterrows():
    if index <= 143:
        continue
    file_id = row['file_ids']
    issue = row['issue_uid']
    status = row['status']
    severity = row['severity']
    issue_url = "https://errata.es-doc.org/static/view.html?uid="+issue
    dfs = pd.DataFrame(columns=keywords)
    print(index, ': issue uid:', issue, ', number of affected files:',len(file_id))
    for file in file_id:
        try:
            [fill,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_version] = file.split('.')
            [grid_label,version] = grid_version.split('#')
            klist = [issue,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,file,status,severity,issue_url]
            kdict = dict(zip(keywords, klist))
            dfs = dfs.append(kdict,ignore_index=True)
            df_all += [dfs]
        except:   # one issue has a list of datasets with two lines that need to be split apart
            #print('splitting lines for:',file_id)
            [s1,s2,s3]=file.split('CMIP6')
            files = ['CMIP6'+s2,'CMIP6'+s3]
            for f in files:
                [fill,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_version] = f.split('.')
                [grid_label,version] = grid_version.split('#')
                klist = [issue,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,f,status,severity,issue_url]
                kdict = dict(zip(keywords, klist))
                dfs = dfs.append(kdict,ignore_index=True)
                df_all += [dfs]
df_expand1 = pd.concat(df_all,sort=False).drop_duplicates(subset =["file_id"])

144 : issue uid: 522ff4de-b4f3-18e6-a892-d6acd1b21847 , number of affected files: 1298
145 : issue uid: 1bf23e20-4ae3-c906-9bd9-669f971ab36b , number of affected files: 158
146 : issue uid: ff701570-b976-b9eb-8b88-348836e14b09 , number of affected files: 27
147 : issue uid: 8fa5880d-208e-d0e1-a0e4-0d0da6668f4b , number of affected files: 12
148 : issue uid: f9fb80b8-9710-c889-b11f-20410dc319d5 , number of affected files: 274
149 : issue uid: f862b568-8b62-8f4f-8f7a-5ebe3f371848 , number of affected files: 105
150 : issue uid: d597592a-1ae2-e8bd-9c31-9a99953eed2b , number of affected files: 158
151 : issue uid: e81e6b1b-f846-27a3-9c43-89770e078810 , number of affected files: 85
152 : issue uid: c1abc1d6-7e83-9249-8c40-70f49e6a1fb6 , number of affected files: 438
153 : issue uid: 4c01d4ab-683d-1ea0-be48-92b52315c608 , number of affected files: 1103
154 : issue uid: fabc475c-b22f-6162-89ca-345d8fdc97ce , number of affected files: 416
155 : issue uid: 4c95f03d-bd45-c583-829c-532c7d433175 ,

In [30]:
df_expand = pd.concat([df_expand0,df_expand1],sort=False).drop_duplicates(subset =["file_id"])
df_expand[['source_id','file_id']].groupby(['source_id']).count()
#df_expand

,file_id
source_id,
ACCESS-CM2,1903
ACCESS-ESM1-5,1605
AWI-CM-1-1-MR,1619
AWI-ESM-1-1-LR,188
CAMS-CSM1-0,105
...,...
NorCPM1,4320
NorESM2-LM,5283
NorESM2-MM,15


In [31]:
import datetime
date = str(datetime.datetime.now().strftime("%Y%m%d"))
os.system('cp csv/errata-files.csv csv/errata-files-'+date+'.csv')
df_expand.to_csv('csv/errata-files.csv', mode='w+', index=False)

In [32]:
import qgrid
qgrid.show_grid(df_expand)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [ ]:
assert False
# rest of notebook is just for testing

In [ ]:
# Make a dataframe to pass to nb1-ReplaceFixed.ipynb - will check 'resolved' tracking_ids and replace if needed

df_fixed = df_expand[df_expand.status=='resolved']
df_fixed['zstore'] = ['gs://cmip6/' + s.split('CMIP6.')[-1][:-9].replace('.','/') + '/' for s in df_fixed.file_id]

zstores= df_fixed.zstore.unique()
len(zstores)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import qgrid
df_fixed.groupby(['source_id','table_id']).count()

In [ ]:
# get dictionary of our existing zstores which may need to be updated
import pickle

with open(f'data/dict.pickle', 'rb') as handle:
    ldict = pickle.load(handle)
len(ldict)

In [ ]:
lfix = {}
for zstore in zstores:
    try:
        lstore = ldict[zstore]
        lfix[zstore]=lstore
    except:
        continue

In [ ]:
keywords = ['activity_id','institution_id','source_id', 'experiment_id', 'member_id', 'table_id', 
            'variable_id', 'grid_label', 'zstore', 'issue_uid']
dfs = pd.DataFrame(columns=keywords)
for zstore, lstore in lfix.items():
    #print(zstore)
    issue = df_fixed.query(f'zstore=="{zstore}"').issue_uid.values[0]
    zarr = zstore.split('gs://cmip6/')[-1][:-1]
    [activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label] = zarr.split('/')
    klist = [activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,issue]
    kdict = dict(zip(keywords, klist))
    dfs = dfs.append(kdict,ignore_index=True)

In [ ]:
date = str(datetime.datetime.now().strftime("%Y%m%d"))
os.system('cp csv/to_fix.csv csv/to_fix-'+date+'.csv')
dfs.to_csv('csv/to_fix.csv',index=False)

In [ ]:
dfs
